<font size=7> Create Raw Datasets

# Create Dataset with bicuculine

## Download raw datasets locally

we download the raw datasets locally using the general command from the `data access` tutorial

In [ ]:
### view folders:
# aws --endpoint https://s3.braingeneers.gi.ucsc.edu s3 ls s3://braingeneers/ephys/2022-11-02-e-Hc11.1-chip16753/
### download metadata
# cd /workspaces/human_hippocampus/data/ephys/2022-11-02-e-Hc11.1-chip16753/
# aws --endpoint https://s3.braingeneers.gi.ucsc.edu s3 cp s3://braingeneers/ephys/2022-11-02-e-Hc11.1-chip16753/metadata.json .
### download raw data
# mkdir -p original/data
# cd original/data
# aws --endpoint https://s3.braingeneers.gi.ucsc.edu s3 cp s3://braingeneers/ephys/2022-11-02-e-Hc11.1-chip16753/original/data/hc11.1_plated-11.1_chip-16753_rec_11.2.22AM-immediate_D.raw.h5 .

Follow data access tutorial on how to load data into python

In [2]:
import braingeneers
from braingeneers import analysis
import braingeneers.data.datasets_electrophysiology as ephys

In [3]:
braingeneers.set_default_endpoint("/workspaces/human_hippocampus/data")
metadata = ephys.load_metadata("2022-11-02-e-Hc11.1-chip16753")

in the metadata we see that experiment 2 is the bicucculine experiment (experiment D)

In [11]:
#metadata["ephys_experiments"].keys()
for key,val in metadata["ephys_experiments"].items():
    print(key)
    print(val["blocks"])
#in the metadata we see that experiment 2 is the bicucculine experiment (experiment D)

experiment3
[{'num_frames': 8368400, 'path': 'original/data/hc11.1_plated-11.1_chip-16753_rec_11.2.22AM-immediate.raw.h5'}]
experiment1
[{'num_frames': 5557600, 'path': 'original/data/hc11.1_plated-11.1_chip-16753_rec_11.2.22AM-immediate_stim.raw.h5'}]
experiment5
[{'num_frames': 3794400, 'path': 'original/data/hc11.1_plated-11.1_chip-16753_rec_11.2.22AM-immediate_stim-B.raw.h5'}]
experiment4
[{'num_frames': 9440800, 'path': 'original/data/hc11.1_plated-11.1_chip-16753_rec_11.2.22AM-immediate_stim-C.raw.h5'}]
experiment2
[{'num_frames': 11353000, 'path': 'original/data/hc11.1_plated-11.1_chip-16753_rec_11.2.22AM-immediate_D.raw.h5'}]
experiment6
[{'num_frames': 9698200, 'path': 'original/data/hc11.1_plated-11.1_chip-16753_rec_11.2.22AM-immediate_E.raw.h5'}]


Load in data from experiment 2

In [15]:
# load in data from experiment 2
raw_data = ephys.load_data( metadata=metadata, experiment="experiment2", offset=5*20000, length=10*20000, channels=None )

/srv/conda/envs/notebook/lib/python3.10/site-packages/braingeneers/data/datasets_electrophysiology.py:209: UserWarning: Metadata is missing the required voltage_scaling_factor attribute. Using default value of 1.0.
  warnings.warn('Metadata is missing the required voltage_scaling_factor attribute. Using default value of 1.0.')


In [16]:
# check the shape of the data
raw_data.shape

(1028, 200000)

## Downsample data

Now that we have loaded the data into python we have select a subset of time from the data for which we want to analyze. For our analysis we don't actually need to consider every 20000 datapoints per second. For this reason, we `downsample` this subset of the data, which means that we consider, say, every other datapoint. This cuts the size of the data in half. In actuallity we on take every 20th datapoint.

In [17]:
from scipy import signal

In [23]:
import numpy as np

function to downsample data

In [18]:
# Downsample abd remove artifact
def downsample(wav_lfp, dec=20, fs=20000.0):
    wav_data = signal.decimate(wav_lfp, dec)
    return fs/dec, wav_data

In [19]:
decimate = 20
fs= 20000
down_fs = fs/decimate

In [24]:
data_down = []  # the variable that will hold the downsambled data
for i in range(len(raw_data)-4): # we run this loop for all code except the last 4 channels, because they are dummy data
    data_down.append( downsample( raw_data[i, 0:(fs*20)], dec=decimate, fs=fs)[1]  ) # we select the first 20seconds of the data, and then downsample it
data_down = np.array( data_down ) # we turn the data into an np.array for easier future analysis

In [25]:
# check the shape of the data
data_down.shape

(1024, 10000)

## Save Data

Save the dataset. We save the data in the folder `ephys_raw` instead of ephys so that the data is available on Github.

<font color="orange">This line is commented out to make sure we don't accidently overwrite the previous saved data

In [26]:
import pickle

In [29]:
#with open( '/workspaces/human_hippocampus/data/ephys_raw/2022-11-02-e-Hc11.1-chip16753/D_start5s_stop15s_fs1ms.pkl' , 'wb') as file:
#    pickle.dump(data_down, file)

check that we can load the data correctly

In [30]:
file = open('/workspaces/human_hippocampus/data/ephys_raw/2022-11-02-e-Hc11.1-chip16753/D_start5s_stop15s_fs1ms.pkl', 'rb')
checkit = pickle.load(file)
file.close()
checkit.shape

(1024, 10000)

# Create  dataset without biccuculine <font color="red"> - Not Done